In [2]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
# Load dataset 
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
sample = pd.read_csv('input/sampleSubmission.csv')
weather = pd.read_csv('input/weather.csv')
# spray = pd.read_csv('input/spray.csv')
# weather station coordinates
station1 = [-87.933 , 41.995]
station2 = [-87.752 , 41.786]

In [15]:
def generate_date_features(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].apply(lambda x: x.year)
    df['Month'] = df['Date'].apply(lambda x: x.month)
    df['DayOfMonth'] = df['Date'].apply(lambda x: x.day)
    # df['DayOfWeekName'] = df['Date'].apply(lambda x: x.day_name())
    df['DayOfWeek'] = df['Date'].apply(lambda x: x.dayofweek)
    df['DayOfYear'] = df['Date'].apply(lambda x: x.dayofyear)
    df['WeekOfYear'] = df['Date'].apply(lambda x: x.weekofyear)
    df['IsLeapYear'] = df['Date'].apply(lambda x: x.is_leap_year)
    df['IsLeapYear'] = df['IsLeapYear'].astype(str)
    df['Quarter'] = df['Date'].apply(lambda x: x.quarter)

def transform_df(df):
    df_ = df.copy()
    df_['Lat_int'] = df_.Latitude.astype(int)
    df_['Long_int'] = df_.Longitude.astype(int)
    return df_

In [23]:
train2 = transform_df(train)
test2 = transform_df(test)

In [24]:
train2.dtypes 

Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
Lat_int                     int64
Long_int                    int64
dtype: object

In [25]:
weather_new = pd.read_csv('weather_new_v1.csv')
weather_new.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Sunrise_time,Sunrise_hour,Sunrise_minute,Sunset_time,Sunset_hour,Sunset_minute
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,29.82,1.7,27,9.2,448,4,48,1849,18,49
1,1,2007-05-02,59,42,51,-3,42,47,14,0,...,30.09,13.0,4,13.4,447,4,47,1850,18,50
2,1,2007-05-03,66,46,56,2,40,48,9,0,...,30.12,11.7,7,11.9,446,4,46,1851,18,51
3,1,2007-05-04,66,49,58,4,41,50,7,0,...,30.05,10.4,8,10.8,444,4,44,1852,18,52
4,1,2007-05-05,66,53,60,5,38,49,5,0,...,30.10,11.7,7,12.0,443,4,43,1853,18,53


In [26]:
from scipy.spatial import distance
def nearest_weather_station(coordinate, station1, station2):
    coordinate_list = coordinate.split(',')
    long = float(coordinate_list[0])
    lat = float(coordinate_list[1])
    current = [long, lat]
    dist1 = distance.euclidean(current,station1)
    dist2 = distance.euclidean(current,station2)
    if dist1 < dist2: return 1
    else: return 2
def transform_data(df, weather_new, station1, station2):
    # get nearest weather
    df_ = df.copy()
    df_['Coordinate'] = df_['Longitude'].astype(str) + "," + df_['Latitude'].astype(str)
    df_['Station'] = df_['Coordinate'].apply(lambda x: nearest_weather_station(x, station1, station2))
    df_with_weather = pd.merge(df_, weather_new, on= ['Date','Station'])
    generate_date_features(df_with_weather) # later
    return df_with_weather

In [27]:
train_with_weather = transform_data(train2, weather_new, station1, station2)
test_with_weather = transform_data(test2, weather_new, station1, station2)

In [28]:
train_with_weather.dtypes

Date                      datetime64[ns]
Address                           object
Species                           object
Block                              int64
Street                            object
Trap                              object
AddressNumberAndStreet            object
Latitude                         float64
Longitude                        float64
AddressAccuracy                    int64
NumMosquitos                       int64
WnvPresent                         int64
Lat_int                            int64
Long_int                           int64
Coordinate                        object
Station                            int64
Tmax                               int64
Tmin                               int64
Tavg                               int64
Depart                             int64
DewPoint                           int64
WetBulb                            int64
Heat                               int64
Cool                               int64
Depth           

In [29]:
# drop address columns
train_with_weather = train_with_weather.drop(['Date','Coordinate','Address', 'AddressNumberAndStreet','WnvPresent', 'NumMosquitos'],  axis = 1)
test_with_weather = test_with_weather.drop(['Date','Coordinate','Id', 'Address', 'AddressNumberAndStreet'],axis = 1)

# label encoder

In [31]:
from sklearn import ensemble, preprocessing

In [39]:
def get_label_encoder(train, test, cols):
    lbl = preprocessing.LabelEncoder()
    for col in cols:
        lbl.fit(list(train[col].values) + list(test[col].values))
        train[col] = lbl.transform(train[col].values)
        test[col] = lbl.transform(test[col].values)

In [40]:
obj_cols = train_with_weather.select_dtypes(include=['object']).columns
obj_cols

Index(['Species', 'Street', 'Trap', 'IsLeapYear'], dtype='object')

In [41]:
get_label_encoder(train_with_weather, test_with_weather, obj_cols)

In [44]:
# Get labels
labels = train.WnvPresent.values

In [50]:
train_data = train_with_weather.values # need to convert to np array

In [52]:
# Random Forest Classifier 
clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=1.0)
clf.fit(train_data, labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=1.0,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [53]:
# create predictions and submission file
# not a good training by itself
predictions = clf.predict(test_with_weather)
sample['WnvPresent'] = predictions 
# wait why is it not binary prediction, all zeros????
# why it is still all 0
sample.WnvPresent.value_counts()

0    116293
Name: WnvPresent, dtype: int64

In [78]:
sample.to_csv('preds_v1_train_only.csv', index=False)